In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [2]:
products = pd.read_csv('/Users/carolinalapid/Desktop/TFM/products_df.csv',index_col=0)

sd = pd.read_csv('/Users/carolinalapid/Desktop/TFM/sociodemographic_df.csv',index_col=0)

cm = pd.read_csv('/Users/carolinalapid/Desktop/TFM/commercial_activity_df.csv',index_col=0)

# Descripción de las variables
### 1. Products
#### Cuentas

em_account_p: Cuenta easyMoney+

em_account_pp: Cuenta easyMoney++

em_acount: Cuenta easyMoney

emc_account: Cuenta easyMoney Crypto

#### payroll: Domiciliaciones

payroll_account: Cuenta bonificada por domiciliaciones

debit_card: Tarjetas de débito


#### Productos de ahorro e inversión

funds: Fondos de inversión

long_term_deposit: Depósitos a largo plazo

pension_plan: Plan de pensiones

securities: Valores

short_term_deposit: Short-term deposits


#### Productos de financiación

credit_card: Tarjetas de crédito

loans: Préstamos

mortgage: hipoteca


### 2. Commercial_activity

entry_channel: Canal de captación del cliente

entry_date: Fecha en la que realizó la primera contratación a través de easyMoney

segment: Segmento comercial del cliente

active_customer: Indicador de actividad del cliente en nuestra aplicación


### 3. Sociodemographic

age: Edad del cliente

country_id: País de residencia del cliente

deceased: Deceased index. N/S

gender: Sexo del cliente

region_code: Provincia de residencia del cliente (para ES)

salary: Ingresos brutos de la unidad familiar

In [3]:
df1= pd.merge(products,cm, how="inner",on=['pk_cid','pk_partition' ])
df=pd.merge(df1,sd, how="inner",on=['pk_cid','pk_partition'])

In [4]:
df.shape

(5962924, 27)

In [5]:
df.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'entry_date',
       'entry_channel', 'active_customer', 'segment', 'country_id',
       'region_code', 'gender', 'age', 'deceased', 'salary'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
 17  entry_date          object 
 18  entry_channel       object 
 19  active_customer     float64
 20  segment             object 
 21  country_id          object 
 22  region_code         float64
 23  gender              object 
 24  age                 int6

In [7]:
df.head(3).T

,0,1,2
pk_cid,1375586,1050611,1050612
pk_partition,2018-01-28,2018-01-28,2018-01-28
short_term_deposit,0,0,0
loans,0,0,0
mortgage,0,0,0
funds,0,0,0
securities,0,0,0
long_term_deposit,0,0,0
em_account_pp,0,0,0
credit_card,0,0,0


In [8]:
dupl_rows = df.duplicated().sum()
print("The number of duplicated rows is:", dupl_rows)

The number of duplicated rows is: 0


In [9]:
df = df.sort_values(by=['pk_cid', 'pk_partition'])

In [12]:
df.columns[df.isnull().any()]

Index(['payroll', 'pension_plan', 'entry_channel', 'segment', 'region_code',
       'gender', 'salary'],
      dtype='object')

In [13]:
## Tratamiento de nulos: Empezamos estudiando las categóricas , para eso dividimos las columnas en función de sus dtypes.

In [14]:
# Identificamos las variables numéricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Identificamos las variables fechas
data_cols = df.select_dtypes(include=['datetime64[ns]']).columns

# Identificamos las variables categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Identificamos las variables booleanas
boolean_cols = df.select_dtypes(include=['bool']).columns

In [15]:
categorical_cols=list(categorical_cols)
data_cols=list(data_cols)
numeric_cols = list(numeric_cols)
boolean_cols = list(boolean_cols)

In [16]:
df.isnull().sum()

pk_cid                      0
pk_partition                0
short_term_deposit          0
loans                       0
mortgage                    0
funds                       0
securities                  0
long_term_deposit           0
em_account_pp               0
credit_card                 0
payroll                    61
pension_plan               61
payroll_account             0
emc_account                 0
debit_card                  0
em_account_p                0
em_acount                   0
entry_date                  0
entry_channel          133033
active_customer             0
segment                133944
country_id                  0
region_code              2264
gender                     25
age                         0
deceased                    0
salary                1512103
dtype: int64

In [17]:
df.value_counts("payroll")

payroll
0.0    5757076
1.0     205787
dtype: int64

In [18]:
products.value_counts("pension_plan")

pension_plan
0.0    5745061
1.0     217802
dtype: int64

In [19]:
df['payroll'].fillna(0.0,inplace=True)
df['pension_plan'].fillna(0.0,inplace=True)

In [20]:
df["entry_channel"].value_counts()

KHE    3113947
KFC     890620
KHQ     590280
KAT     416084
KHK     230197
        ...   
KEJ          8
KHS          5
KFP          2
KDA          2
KDS          1
Name: entry_channel, Length: 68, dtype: int64

In [21]:
df['entry_date'].fillna("sin info",inplace=True)

In [22]:
#Ademas de quitarle los nulos, vamos a agruparla por los principales canales

In [23]:
lista_channels= ["KHE","KFC","KHQ","KAT","KHK"]

In [24]:
lista_channels_menosfrecuente= df["entry_channel"].value_counts().tail(63).index.tolist()

In [25]:
df.loc[df["entry_channel"].isin(lista_channels_menosfrecuente), "entry_channel"]="OTHERS"

In [40]:
df['entry_channel'].value_counts()
df['entry_channel'].fillna("OTHERS",inplace=True)

In [41]:
df["segment"].value_counts()

03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
04 - SIN CATEGORIA     133944
01 - TOP                97939
Name: segment, dtype: int64

In [42]:
df['segment'].fillna('04 - SIN CATEGORIA', inplace= True)

In [43]:
df["gender"].value_counts()

H    3087502
V    2875422
Name: gender, dtype: int64

In [44]:
df['gender'].fillna('V', inplace= True)

In [45]:
salario_segmento =df.groupby('segment')['salary'].mean()
salario_segmento

segment
01 - TOP              148448.090709
02 - PARTICULARES     115887.830537
03 - UNIVERSITARIO    114978.212778
04 - SIN CATEGORIA    113070.572915
Name: salary, dtype: float64

En este caso vemos que particulares y universitarios presentan salarios similares, mientras que los que tienen top presentan una media salarial mayor

La forma con la que imputamos los nulos es en base a la agrupacion que tienen con segment, imputamos los nulos de salary teniendo en cuenta cada uno de los niveles

In [46]:
df['salary'] = df.groupby('segment')['salary'].transform(lambda x: x.fillna(x.mean()))
df['salary'].isnull().sum().sum()

0

In [47]:
df["region_code"].value_counts()

28.0    1188021
8.0      600362
46.0     358142
30.0     296856
41.0     290447
15.0     254891
29.0     187911
36.0     175850
3.0      171358
11.0     160894
33.0     134562
50.0     132983
6.0      122587
35.0     121982
47.0     116948
18.0     102789
45.0     102328
37.0      92538
10.0      83283
14.0      75405
2.0       72071
21.0      72037
13.0      70092
39.0      69589
12.0      68709
7.0       62436
32.0      54639
27.0      52783
43.0      51782
17.0      50749
9.0       47746
25.0      46887
16.0      36094
24.0      35426
48.0      35256
26.0      34790
4.0       32680
31.0      31241
49.0      30965
23.0      30955
38.0      29001
34.0      27545
19.0      26762
40.0      21658
5.0       21300
22.0      21112
20.0      18833
1.0       12999
44.0      12616
42.0       8500
51.0       2896
52.0       2638
Name: region_code, dtype: int64

In [48]:
provincias={1:'Alava',
2:'Albacete',
3:'Alicante',
4:'Almeria',
5:'Avila',
6:'Badajoz',
7:'Baleares',
8:'Barcelona',
9:'Burgos',
10:'Caceres',
11:'Cadiz',
12:'Castellon',
13:'Ciudad Real',
14:'Cordoba',
15:'La Coruna',
16:'Cuenca',
17:'Gerona',
18:'Granada',
19:'Guadalajara',
20:'Guipuzcoa',
21:'Huelva',
22:'Huesca',
23:'Jaen',
24:'Leon',
25:'Lerida',
26:'La Rioja',
27:'Lugo',
28:'Madrid',
29:'Malaga',
30:'Murcia',
31:'Navarra',
32:'Orense',
33:'Asturias',
34:'Palencia',
35:'Las Palmas',
36:'Pontevedra',
37:'Salamanca',
38:'Santa Cruz de Tenerife',
39:'Cantabria',
40:'Segovia',
41:'Sevilla',
42:'Soria',
43:'Tarragona',
44:'Teruel',
45:'Toledo',
46:'Valencia',
47:'Valladolid',
48:'Vizcaya',
49:'Zamora',
50:'Zaragoza',
51:'Ceuta',
52:'Melilla'}

In [49]:
df['provincia']=df['region_code'].map(provincias)

In [50]:
df['provincia'].mode().iloc[0]

'Madrid'

In [51]:
p_mode= df['region_code'].mode().iloc[0]

In [52]:
df['region_code'].fillna(p_mode, inplace=True)

In [53]:
df.isnull().sum()

pk_cid                0
pk_partition          0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
em_account_pp         0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0
entry_date            0
entry_channel         0
active_customer       0
segment               0
country_id            0
region_code           0
gender                0
age                   0
deceased              0
salary                0
provincia             0
dtype: int64

In [54]:
#Ahora que no tenemos nulos, pasaremos a mirar cuales clientes son nuevos:

In [57]:
#corregimos los 29 de febrero y los reemplazamos por 28 de febrero y transformamos el tipo de variable a datetype

In [56]:
df.loc[ (df['entry_date']=='2019-02-29'), 
              'entry_date']='2019-02-28'
df.loc[ (df['entry_date']=='2015-02-29'), 
              'entry_date']='2015-02-28'

In [58]:
for i in ["pk_partition","entry_date"]:
    df[i]=pd.to_datetime(df[i], format='%Y-%m-%d')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 1479563 to 5679912
Data columns (total 28 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int64         
 3   loans               int64         
 4   mortgage            int64         
 5   funds               int64         
 6   securities          int64         
 7   long_term_deposit   int64         
 8   em_account_pp       int64         
 9   credit_card         int64         
 10  payroll             float64       
 11  pension_plan        float64       
 12  payroll_account     int64         
 13  emc_account         int64         
 14  debit_card          int64         
 15  em_account_p        int64         
 16  em_acount           int64         
 17  entry_date          datetime64[ns]
 18  entry_channel       object        
 19  active_customer     float64       
 

In [60]:
#Asigna a una nueva columna llamada 'meses_alta' en el DataFrame. Cada fila de esta columna contiene el número de meses que han transcurrido desde la fecha de entrada hasta la fecha de referencia para ese registro en particular.

In [61]:
df['meses_alta']=(df['pk_partition']- df['entry_date'])/np.timedelta64(1,'M')

In [64]:
df['New_Client']=((df['meses_alta'] < 1) & 
                          (df['meses_alta'] > 0)).astype(int)

In [65]:
df.head(3)

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,...,segment,country_id,region_code,gender,age,deceased,salary,provincia,meses_alta,New_Client
1479563,15891,2018-07-28,0,0,0,0,0,0,0,0,...,04 - SIN CATEGORIA,ES,28.0,H,59,N,113070.572915,Madrid,0.000000,0
2168122,15891,2018-08-28,0,0,0,0,0,0,0,0,...,02 - PARTICULARES,ES,28.0,H,59,N,115887.830537,Madrid,1.018501,0
2962973,16063,2018-11-28,0,0,0,0,0,0,0,0,...,04 - SIN CATEGORIA,ES,28.0,H,62,N,113070.572915,Madrid,0.295694,1


In [66]:
#exportamos el df limpio y con la informacion de new clients

In [67]:
df_clean = df.copy()

In [68]:
df_clean.to_csv("/Users/carolinalapid/Desktop/TFM/df_clean.csv")

Creamos un DF nuevo para analizar las ventas del ultimo mes y poder visualizar cuantos productos hemos vendido este mes o si son los clientes nuevos quiénes los compran

In [85]:
df_1= df[['entry_date','pk_partition', 'segment','short_term_deposit','loans','mortgage','funds','securities','long_term_deposit','em_account_pp','credit_card','payroll','pension_plan','payroll_account','emc_account',
          'debit_card', 'em_account_p','em_acount' ]]
df_1.head()

,entry_date,pk_partition,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
1479563,2018-07-28,2018-07-28,04 - SIN CATEGORIA,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2168122,2018-07-28,2018-08-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
2962973,2018-11-19,2018-11-28,04 - SIN CATEGORIA,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
3628236,2018-11-19,2018-12-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4028169,2018-11-19,2019-01-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0


In [87]:
df_1.groupby('entry_date').sum().reset_index()
ventas = df_1.groupby('entry_date').sum().reset_index()

In [96]:
ventas.head()

,entry_date,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,2015-01-01,0,0,0,0,0,0,0,17,0.0,0.0,0,0,0,0,0
1,2015-01-02,0,0,0,0,16,38,0,195,190.0,204.0,323,674,438,0,1870
2,2015-01-03,0,0,0,41,53,126,0,100,120.0,120.0,239,502,387,0,899
3,2015-01-04,0,0,0,0,33,44,0,125,102.0,116.0,176,558,289,0,577
4,2015-01-05,0,0,0,0,17,33,0,64,87.0,87.0,152,381,222,0,313


In [88]:
#ahora que tenemos las ventas de todos los productos por fecha las exportamos en un csv

ventas.to_csv("/Users/carolinalapid/Desktop/TFM/ventas.csv")

In [94]:
ventas1 = df_1.groupby('pk_partition').sum().reset_index()

In [95]:
ventas1.head()

,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,2018-01-28,883,19,15,786,932,4884,0,3325,8145.0,8835.0,13478,15320,24696,2,215293
1,2018-02-28,1343,19,15,872,965,4956,0,3282,8889.0,9496.0,14246,15827,25448,2,217098
2,2018-03-28,1664,23,17,969,965,4965,0,3511,9735.0,9882.0,14989,16428,27046,2,218683
3,2018-04-28,1740,24,17,1084,983,5107,0,3725,9905.0,10560.0,15825,16947,27764,2,219816
4,2018-05-28,1455,27,18,1145,1012,5356,0,3797,9994.0,10148.0,16697,17569,27911,2,221291


In [97]:
ventas1.to_csv("/Users/carolinalapid/Desktop/TFM/ventas1.csv")

In [98]:
#Creamos una variable de total_products

In [99]:
productos_easymoney= ['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [101]:
df_prod_total = df_1.melt(id_vars=['pk_partition'],
              value_vars = productos_easymoney,
              var_name ='Product',
              value_name ='Count')

In [103]:
df_prod_total.tail()

,pk_partition,Product,Count
83480931,2019-05-28,em_acount,0.0
83480932,2019-05-28,em_acount,0.0
83480933,2019-05-28,em_acount,0.0
83480934,2019-05-28,em_acount,0.0
83480935,2019-05-28,em_acount,0.0


In [104]:
df_prod_total =df_prod_total.groupby(['pk_partition']).agg({'Count':np.sum}).reset_index(drop=False)

In [105]:
df_prod_total.tail()

,pk_partition,Count
12,2019-01-28,411164.0
13,2019-02-28,421404.0
14,2019-03-28,428651.0
15,2019-04-28,432663.0
16,2019-05-28,438696.0


In [106]:
df_prod_total.rename(columns={'Count':'total_productos'}, inplace=True)

In [107]:
len(df_1),len(df_prod_total)

(5962924, 17)

In [108]:
df_final=pd.merge(df_1,df_prod_total, how="inner",on=['pk_partition'])


In [110]:
df_final.tail()

,entry_date,pk_partition,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,total_productos
5962919,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,321043.0
5962920,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962921,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962922,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0
5962923,2018-06-30,2018-06-28,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,321043.0


In [111]:
df_final['total_productos'].value_counts().to_frame()

,total_productos
438696.0,442995
432663.0,439627
428651.0,436183
421404.0,431727
411164.0,426875
411614.0,422481
401801.0,416387
389961.0,402300
367786.0,375323
347677.0,352922


In [115]:
df_final.to_csv("/Users/carolinalapid/Desktop/TFM/df_final.csv")